In [9]:
# Initial imports
from alpha_vantage.timeseries import TimeSeries
import panel as pn
from panel.interact import interact
pn.extension('plotly')
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import hvplot.pandas
import json
import matplotlib.pyplot as plt
from io import StringIO
import csv
from pathlib import Path

In [10]:
# api=os.getenv('ALPHAVANTAGE_API_KEY')

# Supporting Functions

In [11]:
def import_data(ticker):
    """
    A function that imports historical data using Yahoo Finance's hidden API.
    """
    stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?'
    params = {
        'range': '10y',
        'interval': '1d',
        'events': 'history',
        'includeAdjustedClose': 'true'
    }
    response = requests.get(stock_url.format(tickers), params=params)
    file = StringIO(response.text)
    reader = csv.reader(file)
    data = list(reader)

    return pd.DataFrame(data[1:],columns=data[0])


def create_quarters(data):
    """
    creating new columns: year and quarter.
    """
    data['year'] = data['date'].dt.year
    data['quarter'] = data['date'].dt.quarter

def data_groupby_y(data):
    """
    grouping data by year.
    """
    return data.groupby(['year']).mean().round(2).drop(columns=['quarter'])

def data_groupby_q(data):
    """
    grouping data by quarter.
    """
    return data.groupby(['year','quarter']).mean().round(2).reset_index()

def create_hvplot_q(data,title):
    """
    creating an hv line chart for quarterly prices.
    """
    return data.set_index('quarter').hvplot(xlabel='Quarter',ylabel='Closing Price',groupby='year',title=title)

def create_hvplot_y(data,title):
    """
    creating an hv line chart for daily prices.
    """
    return data.drop(columns=['quarter']).set_index('date').hvplot(xlabel='Day',ylabel='Closing Price',groupby='year',title=title)

# Importing Data

In [12]:
file_path = Path("data/gold.csv")
gold_data = pd.read_csv(file_path)
file_path = Path("data/crude_oil.csv")
crude_data = pd.read_csv(file_path)
file_path = Path("data/treasure_index.csv")
treasure_data = pd.read_csv(file_path)
file_path = Path("data/bitcoin.csv")
bitcoin_data = pd.read_csv(file_path)
file_path = Path("data/cpi.csv")
cpi_data = pd.read_csv(file_path)
file_path = Path("data/bloomberg.csv")
bloomberg_data = pd.read_csv(file_path)

In [13]:
ts = TimeSeries(key='6VK2L53MGK88LH76', output_format='pandas')
data, meta_data = ts.get_daily_adjusted(symbol='SPY', outputsize='full')
sp_data = data[['4. close']]
sp_data.head()

,4. close
date,
1999-11-01,135.562500
1999-11-02,134.593704
1999-11-03,135.500000
1999-11-04,136.531204
1999-11-05,137.875000


# Data Cleaning

In [14]:
# Removing first 35 rows(empty from Capital IQ).
gold_data = gold_data.iloc[35:]
crude_data = crude_data.iloc[35:]
treasure_data = treasure_data.iloc[35:]
cpi_data = cpi_data.iloc[35:]
bloomberg_data = bloomberg_data.iloc[35:]

In [15]:
# rename columns
gold_data.columns=['date','gold price']
crude_data.columns=['date','oil price']
treasure_data.columns=['date','treasure index price']
cpi_data.columns=['date','cpi price']
bloomberg_data.columns=['date','bloomberg price']
bitcoin_data.columns=['date','bitcoin price','','','','','']
bitcoin_data = bitcoin_data[['date','bitcoin price']]
sp_data.rename(columns={'4. close': 'S&P 500'},inplace=True)

In [16]:
# Removing commas from bitcoin price
bitcoin_data['bitcoin price']=bitcoin_data['bitcoin price'].str.replace(',','')

In [17]:
# Change data into correct date types
gold_data['date'] = pd.to_datetime(gold_data['date'])
crude_data['date'] = pd.to_datetime(crude_data['date'])
treasure_data['date'] = pd.to_datetime(treasure_data['date'])
cpi_data['date'] = pd.to_datetime(cpi_data['date'])
bloomberg_data['date'] = pd.to_datetime(bloomberg_data['date'])
bitcoin_data['date'] = pd.to_datetime(bitcoin_data['date'])

gold_data['gold price']= gold_data['gold price'].astype(float)
crude_data['oil price']= crude_data['oil price'].astype(float)
treasure_data['treasure index price']= treasure_data['treasure index price'].astype(float)
cpi_data['cpi price'] = cpi_data['cpi price'].astype(float)
bloomberg_data['bloomberg price'] = bloomberg_data['bloomberg price'].astype(float)
bitcoin_data['bitcoin price'] = bitcoin_data['bitcoin price'].astype(float)

In [18]:
# saving datasets
gold_data.set_index('date').to_csv('data/gold_data.csv')
crude_data.set_index('date').to_csv('data/crude_data.csv')
cpi_data.set_index('date').to_csv('data/cpi_data.csv')
bitcoin_data.set_index('date').to_csv('data/bitcoin_data.csv')
treasure_data.set_index('date').to_csv('data/treasure_data.csv')
bloomberg_data.set_index('date').to_csv('data/bloomberg_data.csv')

# Merging Datasets

In [19]:
join_gold = pd.merge(bitcoin_data,gold_data,on='date')
# join_cpi = pd.merge(cpi_data,bitcoin_data,on='date')
# join_crude = pd.merge(bitcoin_data,crude_data,on='date')
# join_bloom = pd.merge(bitcoin_data,bloomberg_data,on='date')
# join_treasure = pd.merge(bitcoin_data,treasure_data,on='date')
# join_all = pd.merge(gold_data, bitcoin_data, on="date").merge(cpi_data, on='date').merge(crude_data, on='date').set_index('date')

In [20]:
# create year and quarter values
create_quarters(join_gold)
# create_quarters(join_cpi)
# create_quarters(join_bloom)
# create_quarters(join_treasure)
# create_quarters(join_crude)

In [21]:
# compute quarterly prices
join_gold_quarter=data_groupby_q(join_gold)
# join_crude_quarter=data_groupby_q(join_crude)
# join_cpi_quarter=data_groupby_q(join_cpi)
# join_treasure_quarter=data_groupby_q(join_treasure)
# join_bloom_quarter=data_groupby_q(join_bloom)

# Visualization

In [22]:
# compare daily prices of bitcoin and gold
create_hvplot_y(join_gold,'Gold VS Bitcoin')

:DynamicMap   [year]
   :NdOverlay   [Variable]
      :Curve   [date]   (value)

In [23]:
# compare quarterly prices of bitcoin and gold
create_hvplot_q(join_gold,'Gold VS Bitcoin')

:DynamicMap   [year]
   :NdOverlay   [Variable]
      :Curve   [quarter]   (value)

In [24]:
join_gold.drop(columns=['quarter','year']).set_index('date').hvplot(xlabel='Day',ylabel='Closing Price',title='Gold VS Bitcoin')

:NdOverlay   [Variable]
   :Curve   [date]   (value)